In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Lectura de datos
ipc_data = pd.read_csv("IPC-2010.csv", parse_dates=[0], infer_datetime_format=True)
ipc_data

In [ ]:
# Normalización de datos con arreglo de numpy con la serie normalizada
ipc_norm = ipc_data["IPC"].values / 100
ipc_norm

In [ ]:
# Número de observaciones
len(ipc_norm)

In [ ]:
# Convertir la serie a un conjunto X, Y de entrenamiento 
X_data = []
Y_data = []
T = len(ipc_norm)
#
# 
# 
# 
# 

In [ ]:
print('Shape datos X de entrenamiento: ', X_data.shape)
print('Shape datos Y de entrenamiento: ', Y_data.shape)

## Definición de la red neuronal de pronóstico

In [ ]:
# Importamos el módulo keras del framework TensorFlow
import tensorflow.keras as keras
import tensorflow as tf
#

In [ ]:
# Importamos el modelo Sequential y las capas Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Dense?

In [ ]:
# np.random.seed(1618)
# tf.random.set_seed(314)

In [ ]:
# Definición del model AR de redes neuronales 
# 
# 
# 
# 

# Este método muestra un resumen de las capas y los parámetros entrenables del modelo
# 

In [ ]:
# Compilación del modelo
# - Definimos la función de pérdida, computada entre la salida del modelo y los valores Y provistos
# - Seleccionamos un algoritmo de optimización
# 
# arnn_model.optimizer.learning_rate.assign(0.001)

In [ ]:
# Llevamos a cabo el entrenamiento
history = arnn_model.fit(x=X_data, \
    y=Y_data, \
    batch_size=32, epochs=100, verbose=0, \
    validation_split=0.15)
print('Final train loss: %0.5f' % history.history["loss"][-1])
print('Final val loss: %0.5f' % history.history["val_loss"][-1])

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')

In [ ]:
# Obtenemos valores de pronóstico intramuestra del modelo
Y_hat = arnn_model.predict(X_data)
Y_hat.shape

In [ ]:
ipc_hat = ipc_data.loc[12:]
ipc_hat["IPC_norm_hat"] = Y_hat*100
ipc_hat

In [ ]:
# Gráfica de los valores ajustados y observados dentro de la muestra
plt.figure(figsize=(10,8))
plt.plot(ipc_hat["Fecha"], ipc_hat.iloc[:, 1:])

## Generando pronósticos de inflación

In [ ]:
# Pronóstico a un horizonte de 12 meses
H = 12
Y_forecast = []
X_forecast_window = ipc_norm[-12:].reshape(1, -1)

#
#
#
#
#

In [ ]:
Y_forecast

In [ ]:
# Creamos rango de fechas
forecast_dates = pd.period_range(start=pd.Period('2020-08', 'M'), periods=H, freq='M')

# Creamos dataframe de pronóstico
ipc_forecast_df = pd.DataFrame({"Fecha": forecast_dates.to_timestamp(), "IPC" : np.array(Y_forecast)*100})
ipc_forecast_df

In [ ]:
# Creamos un DataFrame final con observaciones y pronósticos fuera de muestra
# 
# 

In [ ]:
# Ahora vamos a computar el pronóstico de inflación, para esto utilizamos la columna de IPC
# ipc_final["IPC"][12:]
# ipc_final["IPC"][:-12]

In [ ]:
# Obtener la variación interanual
infl_total = 100 * (ipc_final["IPC"][12:].values / ipc_final["IPC"][:-12].values - 1)
infl_total

In [ ]:
# Agregamos columna de inflación al DataFrame de pronóstico final
ipc_final["Inflacion"] = np.nan
ipc_final["Inflacion"][12:] = infl_total
ipc_final.tail(20)

In [ ]:
# Graficamos los valores de inflación 
plt.figure(figsize=(10,8))
plt.plot(ipc_final["Fecha"], ipc_final["Inflacion"])
plt.ylim((0, 7))

## Guardando el modelo y sus pesos

In [ ]:
# Devuelve una representación en formato JSON del modelo Sequential
arnn_model.to_json()

In [ ]:
# Guardar el modelo en formato JSON
with open("arnn_model.json", "w") as json_file:
    json_file.write(arnn_model.to_json())

# Guardar los pesos de la red neuronal en HDF5
arnn_model.save_weights("arnn_model_weights.h5")
print("Archivos de modelo ARNN generados")

## Modelo con redes neuronales recurrentes de tipo GRU

In [ ]:
# Obtenemos valores de inflación para utilizar como secuencia
seq_inflacion = ipc_final["Inflacion"][12:-12].values

# Convertir la serie a un conjunto X, Y de entrenamiento 
X_data = []
Y_data = []
T = len(seq_inflacion)
for i in range(12, T):
    X_data.append(seq_inflacion[i-12:i])
    Y_data.append(seq_inflacion[i])

X_data = np.array(X_data)
Y_data = np.array(Y_data)

print('Shape datos X: ', X_data.shape)
print('Shape datos Y: ', Y_data.shape)

In [ ]:
np.random.seed(1618)
tf.random.set_seed(314)

In [ ]:
# Definición del modelo para secuencia de valores de inflación 
gru_model = Sequential([
    keras.layers.GRU(32, return_sequences=True, input_shape=[12, 1]),
    keras.layers.GRU(32, return_sequences=False),
    keras.layers.Dense(1, activation='relu')
])

gru_model.summary()

In [ ]:
# gru_model(X_data[:, :, None].astype(np.float32)).shape

In [ ]:
tuned_adam = keras.optimizers.Adam(learning_rate=0.001)
gru_model.compile(optimizer='adam', loss='mse')

In [ ]:
history = gru_model.fit(x=X_data[:, :, None].astype(np.float32), y=Y_data, epochs=100, \
     validation_split=0.1, verbose=1)
print('Final train loss: %0.5f' % history.history["loss"][-1])
print('Final val loss: %0.5f' % history.history["val_loss"][-1])

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')

In [ ]:
# Pronóstico a un horizonte de 12 meses
H = 12
Y_forecast = []
X_forecast_window = seq_inflacion[-12:].reshape(1, -1)

for h in range(1, H+1):
    forecast = gru_model.predict(X_forecast_window[:, :, None])
    Y_forecast.append(forecast.flatten()[0])
    X_forecast_window = np.hstack((X_forecast_window[:, 1:], forecast))

# Creamos rango de fechas
forecast_dates = pd.period_range(start=pd.Period('2020-08', 'M'), periods=H, freq='M')

# Creamos dataframe de pronóstico
infl_forecast_df = pd.DataFrame({"Fecha": forecast_dates.to_timestamp(), "Inflacion" : np.array(Y_forecast)})
infl_forecast_df

In [ ]:
# DataFrame de pronóstico de inflación
infl_final = pd.concat([ipc_final.iloc[12:-12, [0, 2]], infl_forecast_df])
infl_final.tail(20)

In [ ]:
# Graficamos los valores de inflación 
plt.figure(figsize=(10,8))
plt.plot(infl_final["Fecha"], infl_final["Inflacion"])
plt.ylim((0, 7))